# 트위터 연관키워드 분석 (2020년 9월 23일 ~ 2020년 9월 25일)

In [30]:
%matplotlib inline

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

In [32]:
warnings.filterwarnings("ignore")

## 2. 키워드 추출

#### [2-1) 텍스트 데이터 전처리]

#### [2-2) KoNLPy를 이용한 키워드 추출 (형태소 분석기)]
#### 한국어 약식 불용어 사전 사용 (http://ranks.nl/stopwords/korean)

In [33]:
# csv파일 불러오기
df = pd.read_csv('tweet_data_ori_0925.csv')
df.head()

print(df)


from konlpy.tag import Okt
from collections import Counter
from IPython.display import clear_output


korean_stopwords_path = './korean_stopwords.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

nouns_cnt = 1
count_data = []

def get_nouns(x):
    global nouns_cnt, count_data
    
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    count_data.extend([y for y in nouns])
#     count_data.extend([z for y in nouns for z in y])

    #한글자 키워드를 제거합니다.
#     nouns = [noun for noun in nouns if len(noun) > 1]

    #불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]

    clear_output(wait=True)
    print(f'형태소,불용어 제거중 .. {round((nouns_cnt / df.shape[0])*100,3)} %')
    nouns_cnt+=1
    
    return nouns


                        생성일                                                 본문
0       2020-09-23 10:10:53  RT @M2MPD: 44초 짜리 티저 44시간째 돌려보는중..\n  ㄴ 이거 방금 ...
1       2020-09-23 10:10:52  RT @treasuremembers: 여러분...거울에 있는 저만 보세요!!!\n다...
2       2020-09-23 10:10:52  RT @bts_love_myself: 유엔 총회에 참여한 방탄소년단의 메시지를 아래...
3       2020-09-23 10:10:52  RT @iyagideul: [비터애플/BL] 제이윰 작가님의 &lt;여우, 꽃피다&...
4       2020-09-23 10:10:52  RT @hellokoook: 이 짤에서 못나가겠네... https://t.co/zC...
...                     ...                                                ...
337513  2020-09-25 07:06:29    @energydrink1215 @lastm2704 @SaintPorte 살려야 한다.
337514  2020-09-25 07:06:29  흑과 백, 선과 악, 빛과 어둠, 참과 거짓, 이슬람교와 그리스도교, 만남과 헤어짐...
337515  2020-09-25 07:06:28  @L4NYU 그거나쁘지않네요 재밌으면됐죠 지나가는사람들한테 너도술래잡기할래!!하면 ...
337516  2020-09-25 07:06:27  @eto__o 이제 곧 주말인데 모이도 놀아...(폭풍이 휩쓸고 간 자리에서 얼빠진...
337517  2020-09-25 07:06:26                           北  통지문에는\n우리국민 월북 시도 없었다

[337518 rows x 2 columns]


In [34]:
# df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
df = df.iloc[:1000,:]

df['nouns'] = df['본문'].apply(lambda x: get_nouns(x))
print(df.shape)
df.head()

형태소,불용어 제거중 .. 100.0 %
(1000, 3)


,생성일,본문,nouns
0,2020-09-23 10:10:53,RT @M2MPD: 44초 짜리 티저 44시간째 돌려보는중..\n ㄴ 이거 방금 ...,"[티저, 중, 거, 방금, 요]"
1,2020-09-23 10:10:52,RT @treasuremembers: 여러분...거울에 있는 저만 보세요!!!\n다...,"[거울, 건, 볼, 생각, 거울, 속, 사랑, 트, 레저, 메이커]"
2,2020-09-23 10:10:52,RT @bts_love_myself: 유엔 총회에 참여한 방탄소년단의 메시지를 아래...,"[유엔, 총회, 참여, 방탄소년단, 메시지, 아래, 링크, 확인]"
3,2020-09-23 10:10:52,"RT @iyagideul: [비터애플/BL] 제이윰 작가님의 &lt;여우, 꽃피다&...","[비터, 애플, 제이, 윰, 작가, 여우, 꽃피, 전, 권, 간, 기념, 이벤트, ..."
4,2020-09-23 10:10:52,RT @hellokoook: 이 짤에서 못나가겠네... https://t.co/zC...,[짤]


In [6]:
#빈도 수 구하기.

# count_data = [y for x in df['nouns'].tolist() for y in x]

from collections import Counter
count = Counter(count_data)

node_df = pd.DataFrame(count.items(), columns=['node', 'nodesize'])
# node_df = node_df[node_df['nodesize'] >= 50]

# 형태소 데이터 / csv파일로 저장
df.to_csv('save/tmp_data4.csv', index=False)
# 카운트 데이터 / csv파일로 저장
node_df.to_csv('save/tmp_count4.csv', index=False)

node_df.head()

,node,nodesize
0,티저,2
1,중,23
2,거,39
3,방금,3
4,요,22


In [7]:
from ast import literal_eval

# literal_eval이 기본 자료형만
# eval은 모든 스트링을 코드화

df = pd.read_csv('save/tmp_data4.csv',  converters={'nouns': literal_eval})
# df = pd.read_csv('save/tmp_data2.csv', converters={'nouns': eval})
node_df = pd.read_csv('save/tmp_count4.csv')

## 3. 연관 키워드 추출하기

### [트위터 연관 키워드 분석]

In [35]:
transactions = df['nouns'].tolist()
transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.and
# transactions

In [36]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori as apriori2
from mlxtend.frequent_patterns import association_rules

te = TransactionEncoder()
te_result = te.fit(transactions).transform(transactions)

print(te_result.shape[0]*te_result.shape[1])

3628664


In [37]:
df2 = pd.DataFrame(te_result, columns=te.columns_)
df2.head()

,가가,가게,가격,가공,가기,가까이,가끔,가능,가능성,가득,...,희생양,흰색,히스토리,히어로,히어로즈,힉,힐링,힘,힝,힝구
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [45]:
df2 = pd.DataFrame(te_result, columns=te.columns_)

import time
start = time.time()  # 시작 시간 저장


itemset = apriori2(df2, min_support=0.003, use_colnames=True)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

itemset


time : 1.938288688659668


,support,itemsets
0,0.006237,(가능)
1,0.005198,(가면)
2,0.003119,(가수)
3,0.005198,(가야)
4,0.004158,(가을)
...,...,...
1342,0.003119,"(빽투더, 레트로, 쇼, 댄스, 아이돌, 비티)"
1343,0.003119,"(빽투더, 레트로, 쇼, 아이돌, 라이브, 비티)"
1344,0.003119,"(변경, 정보, 주소, 안내, 트위터, 트윗)"
1345,0.003119,"(변경, 정보, 주소, 검색, 트위터, 안내, 트윗)"


In [46]:
itemset2 = association_rules(itemset, metric="confidence", min_threshold=0.0001)
itemset2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(유적),(가야),0.003119,0.005198,0.003119,1.000000,192.400000,0.003102,inf
1,(가야),(유적),0.005198,0.003119,0.003119,0.600000,192.400000,0.003102,2.492204
2,(강아지),(구조),0.004158,0.004158,0.003119,0.750000,180.375000,0.003101,3.983368
3,(구조),(강아지),0.004158,0.004158,0.003119,0.750000,180.375000,0.003101,3.983368
4,(집회),(개천절),0.007277,0.004158,0.004158,0.571429,137.428571,0.004128,2.323631
...,...,...,...,...,...,...,...,...,...
6275,(쇼),"(빽투더, 레트로, 댄스, 아이돌, 라이브, 비티)",0.006237,0.003119,0.003119,0.500000,160.333333,0.003099,1.993763
6276,(댄스),"(빽투더, 레트로, 쇼, 아이돌, 라이브, 비티)",0.003119,0.003119,0.003119,1.000000,320.666667,0.003109,inf
6277,(아이돌),"(빽투더, 레트로, 쇼, 댄스, 라이브, 비티)",0.007277,0.003119,0.003119,0.428571,137.428571,0.003096,1.744543
6278,(라이브),"(빽투더, 레트로, 쇼, 댄스, 아이돌, 비티)",0.009356,0.003119,0.003119,0.333333,106.888889,0.003089,1.495322


In [47]:
# 연관키워드 알고리즘 데이터 csv저장
itemset2.to_csv('save/tmp_last4(0_05).csv', index=False)

In [48]:
# load파일
itemset2 = pd.read_csv('save/tmp_last4(0_05).csv', converters={'antecedents': eval, 'consequents': eval})
itemset2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(유적),(가야),0.003119,0.005198,0.003119,1.000000,192.400000,0.003102,inf
1,(가야),(유적),0.005198,0.003119,0.003119,0.600000,192.400000,0.003102,2.492204
2,(강아지),(구조),0.004158,0.004158,0.003119,0.750000,180.375000,0.003101,3.983368
3,(구조),(강아지),0.004158,0.004158,0.003119,0.750000,180.375000,0.003101,3.983368
4,(집회),(개천절),0.007277,0.004158,0.004158,0.571429,137.428571,0.004128,2.323631


In [50]:
# 조건이 키워드
keyword = '트'
# zz = itemset2[itemset2['antecedents'].apply(lambda x : (keyword in x))]

zz = itemset2[itemset2['antecedents'].apply(lambda x : (keyword in x) & (len(x) == 1))]
zz = zz[zz['consequents'].apply(lambda x : (len(x) == 1))]
# zz = itemset2


#카운트 추가
node_df = pd.read_csv('save/tmp_count4.csv')
def getcount(data) :
    return node_df[node_df['node'].apply(lambda x : data == frozenset({x}))]['nodesize'].values[0]

def getcount_news(data) :
    return df2[next(iter(data))].sum()

# zz['출현_카운트'] = zz['consequents'].apply(lambda x : getcount(x))
# zz['뉴스_카운트'] = zz['consequents'].apply(lambda x : getcount_news(x))

# iat는 한개의 값만 사용할 수 있고 iloc나 loc는 슬라이싱 해서도 사용할 수 있다
# 슬라이싱 하면 결과가 문자값만 나오는게 아니라서 안예쁘지만

# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].iat[0]
# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].iloc[0]
# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].values[0]



pd.set_option('display.max_rows', 1000)
zz.sort_values(by=['lift','support','confidence'], axis='index', ascending=False)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
204,(트),(레저),0.012474,0.010395,0.010395,0.833333,80.166667,0.010265,5.937630
250,(트),(메이커),0.012474,0.007277,0.007277,0.583333,80.166667,0.007186,2.382536
548,(트),(하루),0.012474,0.013514,0.004158,0.333333,24.666667,0.003989,1.479730
348,(트),(사랑),0.012474,0.029106,0.007277,0.583333,20.041667,0.006913,2.330146
414,(트),(앞),0.012474,0.017672,0.004158,0.333333,18.862745,0.003938,1.473493
458,(트),(오늘),0.012474,0.047817,0.005198,0.416667,8.713768,0.004601,1.632314


In [28]:
#숫자가 제거됨.